In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Quimiometria NIR + PLS-DA/PLS-R para Controle de Qualidade Farmacêutico\n",
    "## Dr. José Antônio do Nascimento Neto | JACS Cover Author 2019\n",
    "Modelo real usado em indústria para quantificação de API e detecção de falsificações via Micro-NIR"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.cross_decomposition import PLSRegression\n",
    "from sklearn.model_selection import cross_val_predict\n",
    "from sklearn.metrics import r2_score, mean_squared_error\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "# Dados reais anonimizados de comprimidos (NIR 900–1700 nm)\n",
    "df = pd.read_csv('../data/nir_spectra.csv')\n",
    "X = df.drop(['API_concentration', 'batch'], axis=1)\n",
    "y = df['API_concentration']\n",
    "\n",
    "print(f'Shape: {X.shape} | API range: {y.min():.1f} – {y.max():.1f}%')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# PLS-R com 8 componentes (otimizado por leave-one-out)\n",
    "pls = PLSRegression(n_components=8, scale=True)\n",
    "y_pred = cross_val_predict(pls, X, y, cv=10)\n",
    "\n",
    "r2 = r2_score(y, y_pred)\n",
    "rmse = np.sqrt(mean_squared_error(y, y_pred))\n",
    "\n",
    "plt.figure(figsize=(8,6))\n",
    "sns.scatterplot(x=y, y=y_pred, hue=df['batch'], palette='tab10')\n",
    "plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')\n",
    "plt.title(f'PLS-R NIR → API% | R² = {r2:.3f} | RMSE = {rmse:.2f}%')\n",
    "plt.xlabel('Valor de referência (%)')\n",
    "plt.ylabel('Valor predito PLS (%)')\n",
    "plt.grid(True)\n",
    "plt.savefig('../results/PLS_NIR_API.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}